In [4]:
pip install plotly_express==0.4.0


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install Scrapy

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os 
import logging
import scrapy
import requests
from scrapy.crawler import CrawlerProcess
import json
import pandas as pd
import plotly.express as px
import glob
import boto3

ModuleNotFoundError: No module named 'boto3'

# Cities and creation of df

In [9]:
city_names= ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas"
]

In [10]:
df = pd.DataFrame(columns=['city_id' , 'name', 'latitude', 'longitude', 'main_weather', 'expected_rain', 'day_temperature'])
df

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature


# Coordinates, weather info, convert df to csv

In [11]:
api_key = "f482d2b26c2e58bf480acbe962e74f18"

In [12]:
for i in range(len(city_names)):
    print("making request: {}".format(city_names[i]))
    
#get latitude and longitude of cities    
    
    par = {
        "city": city_names[i],
        "country": "France",
        "format": "json"
          }
    
    r = requests.get('https://nominatim.openstreetmap.org/search', params=par)
    res = r.json()
    
#fix lat and lon
    df.loc[i,'city_id'] = i
    df.loc[i,'name'] = city_names[i]
    df.loc[i, 'latitude'] = res[0]['lat']
    df.loc[i, 'longitude'] = res[0]['lon']
    
#get weather
    par = {
        "lat": df.loc[i, 'latitude'],
        "lon": df.loc[i, 'longitude'],
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": api_key,
           }
    
    r = requests.get('https://api.openweathermap.org/data/2.5/onecall', params=par)
    res = r.json()
    
    # Compute expected volume of rain
    expected_rain = 0
    print(res) 
    
    for d in res['daily']:
        if 'rain' in d.keys():
            expected_rain += d['pop']*d['rain']
            
    # Compute average day temperature
    temperatures = pd.Series([d['temp']['day'] for d in res['daily']])
    mean_temperature = temperatures.mean()

    # Extract most probable weather
    weathers = pd.Series([d['weather'][0]['main'] for d in res['daily']])
    main_weather = weathers.mode()[0]
    
    df.loc[i, 'main_weather'] = main_weather
    df.loc[i,'expected_rain'] = expected_rain
    df.loc[i,'day_temperature'] = mean_temperature


making request: Mont Saint Michel
{'lat': 48.6359, 'lon': -1.5126, 'timezone': 'Europe/Paris', 'timezone_offset': 3600, 'daily': [{'dt': 1610971200, 'sunrise': 1610956254, 'sunset': 1610988102, 'temp': {'day': 7.31, 'min': 3.45, 'max': 8.65, 'night': 6.93, 'eve': 7.39, 'morn': 3.88}, 'feels_like': {'day': 2.69, 'night': -0.22, 'eve': 1.96, 'morn': -0.13}, 'pressure': 1026, 'humidity': 77, 'dew_point': 3.67, 'wind_speed': 4.6, 'wind_deg': 216, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'clouds': 99, 'pop': 0.07, 'uvi': 0.93}, {'dt': 1611057600, 'sunrise': 1611042603, 'sunset': 1611074591, 'temp': {'day': 10.38, 'min': 7.63, 'max': 10.73, 'night': 9.42, 'eve': 9.21, 'morn': 9.63}, 'feels_like': {'day': 3.9, 'night': 1.18, 'eve': 3.08, 'morn': 2.97}, 'pressure': 1013, 'humidity': 83, 'dew_point': 7.75, 'wind_speed': 8.47, 'wind_deg': 202, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': 98, 'p

IndexError: list index out of range

In [13]:
df

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature
0,0,Mont Saint Michel,48.6358559,-1.5126096,Rain,38.8919,8.465
1,1,St Malo,48.6454528,-2.015418,Rain,33.152,8.7875
2,2,Bayeux,49.2764624,-0.7024738,Rain,34.2027,7.755
3,3,Le Havre,49.4938975,0.1079732,Rain,39.7682,7.78625
4,4,Rouen,49.4404591,1.0939658,Rain,29.4202,6.87
5,5,Paris,48.8566969,2.3514616,Rain,29.3093,6.7575
6,6,Amiens,49.8941708,2.2956951,Rain,25.2624,6.50875
7,7,Lille,50.6365654,3.0635282,Rain,17.2693,6.37625
8,8,Strasbourg,48.584614,7.7507127,Rain,12.4086,4.2575
9,9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923,Snow,22.2542,1.25875


In [14]:
df.loc[:,'rank'] = df['expected_rain'].rank(method='min')
df.loc[:,'inverted_rank'] = df['expected_rain'].rank(method='min', ascending=False)
df = df.sort_values(by=['expected_rain', 'day_temperature'], ascending = [True, False]).reset_index(drop=True)
display(df)

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,8,Strasbourg,48.584614,7.7507127,Rain,12.4086,4.2575,1.0,17.0
1,7,Lille,50.6365654,3.0635282,Rain,17.2693,6.37625,2.0,16.0
2,10,Colmar,48.0777517,7.3579641,Snow,22.0231,4.43,3.0,15.0
3,9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923,Snow,22.2542,1.25875,4.0,14.0
4,16,Lyon,45.7578137,4.8320114,Rain,22.98,7.74,5.0,13.0
5,11,Eguisheim,48.0447968,7.3079618,Snow,23.7484,4.51625,6.0,12.0
6,6,Amiens,49.8941708,2.2956951,Rain,25.2624,6.50875,7.0,11.0
7,5,Paris,48.8566969,2.3514616,Rain,29.3093,6.7575,8.0,10.0
8,4,Rouen,49.4404591,1.0939658,Rain,29.4202,6.87,9.0,9.0
9,15,Grenoble,45.1875602,5.7357819,Clouds,29.8511,6.57,10.0,8.0


In [15]:
print('Best places for a trip next week are : ')
print()

for i,row in df.loc[df['rank']==1,:].iterrows():
    print("{} -- Mostly {} with temperature {} °C".format(row['name'], row['main_weather'], row['day_temperature']))

Best places for a trip next week are : 

Strasbourg -- Mostly Rain with temperature 4.2575 °C


In [16]:
df = df.fillna(0)

In [17]:
df.loc[:,'latitude'] = df['latitude'].astype('float')
df.loc[:,'longitude'] = df['longitude'].astype('float')
df.loc[:,'expected_rain'] = df['expected_rain'].astype('float')
df.loc[:,'day_temperature'] = df['day_temperature'].astype('float')
df.loc[:,'rank'] = df['rank'].astype('int')
df.loc[:,'inverted_rank'] = df['inverted_rank'].astype('int')

In [21]:
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['main_weather', 'expected_rain', 'day_temperature'], 
                        color = 'day_temperature', color_continuous_scale = 'Bluered', size = 'inverted_rank',
                        mapbox_style="carto-positron")
fig.show()

In [23]:
df.to_csv('destinations.csv', index=False)

# Scrapping Booking.com for hotel information

### La Rochelle

In [31]:
city = 'La Rochelle'

In [32]:
r = requests.get('https://www.booking.com/index.fr.html')
r

<Response [200]>

In [33]:
class BookingSpider(scrapy.Spider):
    name = "booking"

    start_urls = ['https://www.booking.com/index.fr.html']

    def parse(self, response):
        return scrapy.FormRequest.from_response(
            response,
            formdata={'ss': city},
            callback=self.after_search
        )

    def after_search(self, response):
        
        booking = response.css('.sr_item')

        for data in booking:
            yield {
                'name': data.css('.sr-hotel__name::text').get(),
                'url': 'https://www.booking.com' + data.css('.hotel_name_link').attrib["href"],
                'coords': data.css('.sr_card_address_line a').attrib["data-coords"],
                'score': data.css('.bui-review-score__badge::text').get(),
                'description': data.css('.hotel_desc::text').get()
                
            }
        
        
        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)

In [35]:
filename = "hotels_" + city.replace(" ", "-") + ".json"


process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'res/' + filename: {"format": "json"},
    }
})

process.crawl(BookingSpider)
process.start()

2021-01-18 16:39:12 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-01-18 16:39:12 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-4.19.112+-x86_64-with-glibc2.10
2021-01-18 16:39:12 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-01-18 16:39:12 [scrapy.extensions.telnet] INFO: Telnet Password: 3ece43f321671dd3
2021-01-18 16:39:12 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-01-18 16:39:12 [scrapy.middleware] INFO: Ena

ReactorNotRestartable: 

### Paris

In [7]:
city = 'Paris'

In [8]:
r = requests.get('https://www.booking.com/index.fr.html')
r

<Response [200]>

In [9]:
class BookingSpider(scrapy.Spider):
    name = "booking"

    start_urls = ['https://www.booking.com/index.fr.html']

    def parse(self, response):
        return scrapy.FormRequest.from_response(
            response,
            formdata={'ss': city},
            callback=self.after_search
        )

    def after_search(self, response):
        
        booking = response.css('.sr_item')

        for b in booking:
            yield {
                'name': b.css('.sr-hotel__name::text').get(),
                'url': 'https://www.booking.com' + b.css('.hotel_name_link').attrib["href"],
                'coords': b.css('.sr_card_address_line a').attrib["data-coords"],
                'score': b.css('.bui-review-score__badge::text').get(),
                'description': b.css('.hotel_desc::text').get()
                
            }
        
        
        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)

In [10]:
filename = "hotels_" + city.replace(" ", "-") + ".json"


process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'res/' + filename: {"format": "json"},
    }
})

process.crawl(BookingSpider)
process.start()

2021-01-18 16:41:08 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-01-18 16:41:08 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-4.19.112+-x86_64-with-glibc2.10
2021-01-18 16:41:08 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-01-18 16:41:08 [scrapy.extensions.telnet] INFO: Telnet Password: d3cbb497100a08a0
2021-01-18 16:41:08 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-01-18 16:41:08 [scrapy.middleware] INFO: Ena

# Data lake with S3

In [14]:
cities = pd.read_csv('destinations.csv')
cities.head()

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,8,Strasbourg,48.584614,7.750713,Rain,12.4086,4.25750,1,17
1,7,Lille,50.636565,3.063528,Rain,17.2693,6.37625,2,16
2,10,Colmar,48.077752,7.357964,Snow,22.0231,4.43000,3,15
3,9,Chateau du Haut Koenigsbourg,48.249523,7.345492,Snow,22.2542,1.25875,4,14
4,16,Lyon,45.757814,4.832011,Rain,22.9800,7.74000,5,13


In [39]:
hotel_files = glob.glob('res/_hotels_*.json')

hotels = pd.DataFrame(columns = ['city_id', 'city_name', 'hotel_id', 'name', 'url', 'latitude', 'longitude', 'score', 'description'])
hotels


,city_id,city_name,hotel_id,name,url,latitude,longitude,score,description


In [40]:
for f in hotel_files:
    city_name = f.split('_')[2].split('.')[0].replace("-"," ")

In [111]:
for f in hotel_files:
    city_id = cities.loc[cities['name'] == city_name,'city_id'].values[0]

In [112]:
hotel_files

['res/_hotels_Paris.json']

In [160]:
f1 = pd.read_json('res/hotels_Paris.json')
hotels=pd.DataFrame(f1)
hotels['name']=(hotels['name']).astype(str).replace('\n','',regex=True)
hotels['description']=(hotels['description']).astype(str).replace('\n','',regex=True)
hotels.head()

,name,url,coords,score,description
0,Elysées Union,https://www.booking.com\n/hotel/fr/elyseesunio...,"2.29176588356495,48.8683494219935","8,4","Occupant un bâtiment du XIXe siècle à Paris, l..."
1,Hyatt Regency Paris Etoile,https://www.booking.com\n/hotel/fr/hyatt-regen...,"2.28472741799624,48.8804098381377","8,4","Le Hyatt Regency Paris Etoile, connu autrefois..."
2,Hotel Lutetia,https://www.booking.com\n/hotel/fr/lutetia-par...,"2.327238,48.851244","8,9",L’Hotel Lutetia est un établissement de luxe e...
3,Pullman Paris Tour Eiffel,https://www.booking.com\n/hotel/fr/tour-eiffel...,"2.29256182909012,48.855604014139","8,3",L'hôtel 4 étoiles Pullman Paris Tour Eiffel es...
4,Hotel Prelude Opera,https://www.booking.com\n/hotel/fr/corona.fr.h...,"2.34349831938744,48.871934420399","9,0",L'Hotel Corona Opera se situe dans l'élégant 9...


In [161]:
split_data = df.coords.str.split(',')
hotels.loc[:,'latitude'] = split_data.apply(lambda x: x[0])
hotels.loc[:,'longitude'] = split_data.apply(lambda x: x[1])
split_data




0      [-1.15486329021303, 46.1510701196158]
1       [-1.1410117149353, 46.1584466937086]
2            [-1.14855799999998, 46.1558294]
3      [-1.14639222621918, 46.1557694290086]
4      [-1.15847557783127, 46.1614340775011]
                       ...                  
349                   [-1.153158, 46.144291]
350                   [-1.167898, 46.140315]
351                   [-1.168566, 46.140806]
352                    [-1.156527, 46.14409]
353                    [-1.169634, 46.13962]
Name: coords, Length: 354, dtype: object

In [129]:
#hotels_all = hotels_all.append(hotels)

NameError: name 'hotels_all' is not defined

In [165]:
del hotels['coords']
hotels.head()

,name,url,score,description,latitude,longitude
0,Elysées Union,https://www.booking.com\n/hotel/fr/elyseesunio...,"8,4","Occupant un bâtiment du XIXe siècle à Paris, l...",-1.15486329021303,46.1510701196158
1,Hyatt Regency Paris Etoile,https://www.booking.com\n/hotel/fr/hyatt-regen...,"8,4","Le Hyatt Regency Paris Etoile, connu autrefois...",-1.1410117149353,46.1584466937086
2,Hotel Lutetia,https://www.booking.com\n/hotel/fr/lutetia-par...,"8,9",L’Hotel Lutetia est un établissement de luxe e...,-1.14855799999998,46.1558294
3,Pullman Paris Tour Eiffel,https://www.booking.com\n/hotel/fr/tour-eiffel...,"8,3",L'hôtel 4 étoiles Pullman Paris Tour Eiffel es...,-1.14639222621918,46.1557694290086
4,Hotel Prelude Opera,https://www.booking.com\n/hotel/fr/corona.fr.h...,"9,0",L'Hotel Corona Opera se situe dans l'élégant 9...,-1.15847557783127,46.1614340775011


In [166]:
hotels.loc[:, 'city_name']='Paris'
hotels.loc[:, 'city_id'] = cities.loc[cities['name'] == city_name,'city_id'].values[0]
hotels.loc[:, 'latitude'] = hotels['latitude'].astype('float')
hotels.loc[:, 'longitude'] = hotels['longitude'].astype('float')
hotels.loc[:, 'score'] = hotels['score'].str.replace(',','.').astype('float')

In [42]:
hotels.head()

,city_id,city_name,hotel_id,name,url,latitude,longitude,score,description
0,5,Paris,0,Elysées Union,https://www.booking.com/hotel/fr/elyseesunion....,48.868349,2.291766,8.4,"Occupant un bâtiment du XIXe siècle à Paris, l..."
1,5,Paris,1,Hyatt Regency Paris Etoile,https://www.booking.com/hotel/fr/hyatt-regency...,48.880410,2.284727,8.4,"Le Hyatt Regency Paris Etoile, connu autrefois..."
2,5,Paris,2,Pullman Paris Tour Eiffel,https://www.booking.com/hotel/fr/tour-eiffel.f...,48.855604,2.292562,8.3,L'hôtel 4 étoiles Pullman Paris Tour Eiffel es...
3,5,Paris,3,citizenM Paris Gare de Lyon,https://www.booking.com/hotel/fr/citizenm-pari...,48.843726,2.371727,8.7,"Offrant une vue sur la Seine, le citizenM Pari..."
4,5,Paris,4,Hotel Lutetia,https://www.booking.com/hotel/fr/lutetia-paris...,48.851244,2.327238,8.9,L’Hotel Lutetia est un établissement de luxe e...


In [44]:
hotels.to_csv('res/0_hotels.csv', index=False)

In [35]:
hotels_with_score = hotels.loc[hotels['score'].notnull(),:]

fig = px.scatter_mapbox(hotels_with_score, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['description'],
                        color = 'score', color_continuous_scale = 'thermal',
                        mapbox_style="carto-positron")
fig.show()

NameError: name 'hotels' is not defined